<a href="https://colab.research.google.com/github/simplysumanth/dl-with-pytorch-d2l.ai/blob/main/dl_ch5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
from torch import nn as nn
from torch.nn import functional as F

In [16]:
net = nn.Sequential(
    nn.Linear(20,256),
    nn.ReLU(),
    nn.Linear(256,10))

In [14]:
X = torch.rand(2,20)
X.size()

torch.Size([2, 20])

In [18]:
net(X)

tensor([[-0.0934, -0.1850,  0.1932,  0.1837,  0.0577, -0.1174, -0.0829, -0.0458,
          0.0661,  0.1085],
        [ 0.0216, -0.1286,  0.0506,  0.2221, -0.0118, -0.0776, -0.0821, -0.0264,
          0.0786, -0.0148]], grad_fn=<AddmmBackward>)

In [21]:
class MLP(nn.Module):
  def __init__(self):
    super().__init__()
    self.hidden = nn.Linear(20,256)
    self.out = nn.Linear(256,10)
  
  def forward(self,x):
    return self.out(F.relu(self.hidden(X)))

In [22]:
net = MLP()
net(X)

tensor([[-0.3284,  0.3045,  0.1374, -0.0380,  0.0493, -0.1134,  0.1358, -0.2331,
          0.0713,  0.0265],
        [-0.2625,  0.3253,  0.0970,  0.0459,  0.1228, -0.1703,  0.1543, -0.2827,
          0.0287, -0.0181]], grad_fn=<AddmmBackward>)